In [ ]:
import tensorflow as tf

##dataset

In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d subhajournal/phishingemails

mkdir: cannot create directory ‘/root/.kaggle’: File exists
cp: cannot stat '/content/drive/MyDrive/kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/subhajournal/phishingemails
License(s): GNU Lesser General Public License 3.0
phishingemails.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip phishingemails.zip

Archive:  phishingemails.zip
replace Phishing_Email.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
data=pd.read_csv("Phishing_Email.csv")
data.head(10)

,Unnamed: 0,Email Text,Email Type
0,0,"re : 6 . 1100 , disc : uniformitarianism , re ...",Safe Email
1,1,the other side of * galicismos * * galicismo *...,Safe Email
2,2,re : equistar deal tickets are you still avail...,Safe Email
3,3,\nHello I am your hot lil horny toy.\n I am...,Phishing Email
4,4,software at incredibly low prices ( 86 % lower...,Phishing Email
5,5,global risk management operations sally congra...,Safe Email
6,6,"On Sun, Aug 11, 2002 at 11:17:47AM +0100, wint...",Safe Email
7,7,"entourage , stockmogul newsletter ralph velez ...",Phishing Email
8,8,"we owe you lots of money dear applicant , afte...",Phishing Email
9,9,re : coastal deal - with exxon participation u...,Safe Email


In [ ]:
email_texts=data["Email Text"].astype(str)
email_labels=data["Email Type"]
email_labels

,Email Type
0,Safe Email
1,Safe Email
2,Safe Email
3,Phishing Email
4,Phishing Email
...,...
18645,Phishing Email
18646,Safe Email
18647,Safe Email
18648,Safe Email


##preprocessing

In [ ]:
unique=np.unique(email_labels)
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
email_labels=le.fit_transform(email_labels)

In [ ]:
email_labels

array([1, 1, 1, ..., 1, 1, 0])

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    email_texts, email_labels, test_size=0.2, random_state=42
)
train_labels = pd.Series(train_labels)
val_labels = pd.Series(val_labels)

# Reset indices after splitting
train_texts = train_texts.reset_index(drop=True)
val_texts = val_texts.reset_index(drop=True)
train_labels = train_labels.reset_index(drop=True)
val_labels = val_labels.reset_index(drop=True)

tokenizer=Tokenizer(num_words=10000,oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)

batch_size = 32

# Maximum sequence length
max_sequence_length = 50

# Data generator (same as before)
def data_generator(texts, labels, batch_size, tokenizer, maxlen=None):
  """Generates batches of padded sequences."""
  num_samples = len(texts)
  indices = np.arange(num_samples)
  while True:
    np.random.shuffle(indices)  # Shuffle data for each epoch
    for i in range(0, num_samples, batch_size):
      batch_indices = indices[i: min(i + batch_size, num_samples)]
      batch_texts = texts[batch_indices].astype(str)
      batch_sequences = tokenizer.texts_to_sequences(batch_texts)
      batch_padded = pad_sequences(batch_sequences, padding="post", maxlen=maxlen)
      batch_labels = labels[batch_indices]
      yield batch_padded, batch_labels

model2=tf.keras.Sequential([
    tf.keras.layers.Embedding(10000,128,input_length=max_sequence_length),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1,activation="sigmoid")
])
model2.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

train_generator = data_generator(
    train_texts, train_labels, batch_size, tokenizer, maxlen=max_sequence_length
)
val_generator = data_generator(
    val_texts, val_labels, batch_size, tokenizer, maxlen=max_sequence_length
)
# Calculate steps per epoch
train_steps_per_epoch = len(train_texts) // batch_size
val_steps_per_epoch = len(val_texts) // batch_size

# Train the model
epochs = 10  # Adjust as needed
model2.fit(
    train_generator,
    steps_per_epoch=train_steps_per_epoch,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_steps_per_epoch,
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


466/466 ━━━━━━━━━━━━━━━━━━━━ 12s 23ms/step - accuracy: 0.8151 - loss: 0.3910 - val_accuracy: 0.9475 - val_loss: 0.1409
Epoch 2/10
466/466 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - accuracy: 0.9623 - loss: 0.1202 - val_accuracy: 0.9429 - val_loss: 0.1499
Epoch 3/10
466/466 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - accuracy: 0.9695 - loss: 0.0743 - val_accuracy: 0.9397 - val_loss: 0.1790
Epoch 4/10
466/466 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - accuracy: 0.9771 - loss: 0.0552 - val_accuracy: 0.9540 - val_loss: 0.1449
Epoch 5/10
466/466 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - accuracy: 0.9822 - loss: 0.0422 - val_accuracy: 0.9575 - val_loss: 0.1603
Epoch 6/10
466/466 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - accuracy: 0.9860 - loss: 0.0321 - val_accuracy: 0.9567 - val_loss: 0.1502
Epoch 7/10
466/466 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.9856 - loss: 0.0313 - val_accuracy: 0.9532 - val_loss: 0.1486
Epoch 8/10
466/466 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.9880 - loss: 0.0245 - val_accuracy: 

In [ ]:
val_texts, test_texts, val_labels, test_labels=train_test_split(train_texts, train_labels, test_size=0.2, random_state=42)

test_sequences = tokenizer.texts_to_sequences(test_texts)
test_padded = pad_sequences(test_sequences, padding="post", maxlen=max_sequence_length)
test_labels = np.array(test_labels)

model.evaluate(test_padded,test_labels)

94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9843 - loss: 0.0359


[0.034664336591959, 0.9865951538085938]